In [74]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from tableauscraper import TableauScraper as TS

### Download the tableau dashboard

In [75]:
url = "https://public.tableau.com/views/CaliforniaUrbanWaterProduction_15785959527960/StatewideUseandProduction"

In [76]:
ts = TS()
ts.loads(url)
workbook = ts.getWorkbook()

In [77]:
for t in workbook.worksheets:
    if t.name == "RGPCD_HYDRO":
        total_hydro_df = t.data
    elif t.name == "Total_Production":
        total_df = t.data
    else:
        pass

In [78]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [79]:
df_list = []
for m in months:
    wb = ws.setFilter('MONTH(Reporting Month)', m)
    monthlyWs = wb.getWorksheet('RGPCD_HYDRO')
    monthly_df = monthlyWs.data
    monthly_df["month"] = m
    df_list.append(monthly_df)

2022-05-18 19:09:59,763 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:09:59,763 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:09:59,763 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:00,454 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:00,454 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:00,454 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:01,121 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:01,121 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:01,121 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:01,886 - tableauScraper - WARNING - no data dictionary present in response
2022-05-18 19:10:01,886 - tableauScraper - WARNING - no data dictionary present 

In [80]:
df = pd.concat(df_list)

In [81]:
# for region in list(df["Region-value"].unique()):
#     df_filter = df[df["Region-value"]==region]
#     print(len(df_filter))

In [82]:
pivot = df.pivot(index=['Region-value','month'], columns='YEAR(Reporting Month)-alias', values='AGG(Pop_Weighted_RGPCD)-alias').reset_index()

In [83]:
trim_cols = ['Region-value', 'AGG(NumReporting)-alias', 'month',
       'YEAR(Reporting Month)-alias',
       'AGG(Pop_Weighted_RGPCD)-alias']

In [84]:
df_trim = df[trim_cols]

In [85]:
df_rename = df_trim.rename(columns={
    'Region-value':'hydrologic_region', 
    'AGG(NumReporting)-alias': 'agencies_reporting', 
    'YEAR(Reporting Month)-alias': 'year',
    'AGG(Pop_Weighted_RGPCD)-alias': 'pop_weighted_rgpcd'
})

In [86]:
df_rename.loc[(df_rename.hydrologic_region == "%all%"), "hydrologic_region"] = "Statewide"

In [87]:
df_rename.to_csv("data/raw/monthly-water-use-by-region.csv", index=False)